### Runnable

- Runnable은 런타임에 실행될 수 있는 모든 객체를 의미한다.

### Runnable 종류
- invoke : 입력에 대해 체인을 호출하는 메소드
- stream : 응답의 청크를 스트리밍한다.
(응답을 생성될 때마다 하나하나 받아줘서 한 응답으로 연결해서 보여주는 것)
- batch : 입력 목록에 대한 일괄 처리를 수행한다.

이외에 비동기 메소드도 존재한다.
* astream : 비동기적으로 응답의 청크를 비동기 스트리밍한다.
* ainvoke : 비동기적으로 입력에 대해 체인을 호출한다.
* abatch : 비동기적으로 입력 목록에 대해 일괄 처리를 수행한다.
* astream_log : 최종 응답 및 발생하는 중간단계를 스트리밍한다.

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.1
)

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{lecture}에 대해 3문장으로 설명해줘")

chain = prompt | llm | StrOutputParser()

In [13]:
print(chain.invoke({"lecture":"리엑트"}))

리액트(React)는 사용자 인터페이스를 구축하기 위한 JavaScript 라이브러리로, 주로 단일 페이지 애플리케이션(SPA) 개발에 사용됩니다. 컴포넌트 기반 아키텍처를 통해 재사용 가능한 UI 구성 요소를 만들 수 있으며, 가상 DOM을 사용하여 성능을 최적화합니다. 리액트는 페이스북에 의해 개발되었으며, 커뮤니티와 생태계가 활발하여 다양한 도구와 라이브러리와 함께 사용할 수 있습니다.


### stream[실시간 출력]
- 데이터 스트림을 생성하고, 스트림을 반복하여 각 데이터의 내용을 즉시 출력한다.
- end="" 인자를 사용해 줄바꿈을 하지 않을 수 있다.
- flush=True 인자를 사용해 즉시 출력 가능

In [14]:
for token in chain.stream({"lecture":"Python"}):
    print(token, end="", flush=True)

Python은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 용도로 사용됩니다. 데이터 분석, 웹 개발, 인공지능 등 여러 분야에서 널리 활용되며, 방대한 라이브러리와 커뮤니티 지원이 특징입니다. 또한, 플랫폼 독립적이어서 다양한 운영 체제에서 실행할 수 있는 장점이 있습니다.

### batch [단위 실행]
- 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각각의 입력에 대한 체인을 실행한다.
- max_concurrency : 최대 병렬 처리 수를 지정할 수 있다.

In [21]:
result = chain.batch(
    [
        {"lecture":"파이썬"},
        {"lecture":"자바"},
        {"lecture":"자바스크립트"},
        {"lecture":"스프링"}
    ],
    config={"max_concurrency" : 3}
)

print(result)

['파이썬은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 분야에서 널리 사용됩니다. 데이터 분석, 웹 개발, 인공지능 등 여러 응용 프로그램을 지원하는 풍부한 라이브러리와 프레임워크를 제공합니다. 또한, 플랫폼 독립적이며 커뮤니티가 활발하여 학습 자료와 지원이 풍부합니다.', '자바는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 제공하여 "한 번 작성하면 어디서나 실행"할 수 있는 특징이 있습니다. 강력한 메모리 관리와 자동 가비지 수집 기능을 통해 개발자가 메모리 관리를 신경 쓰지 않아도 되도록 돕습니다. 또한, 풍부한 라이브러리와 프레임워크를 통해 다양한 애플리케이션 개발에 널리 사용됩니다.', '자바스크립트는 웹 페이지에 동적인 기능을 추가하기 위해 사용되는 프로그래밍 언어입니다. 클라이언트 측에서 실행되며, HTML과 CSS와 함께 웹 개발의 핵심 기술 중 하나로 자리잡고 있습니다. 또한, Node.js와 같은 런타임 환경을 통해 서버 측 프로그래밍에도 활용될 수 있습니다.', '스프링(Spring)은 자바 플랫폼을 위한 오픈 소스 애플리케이션 프레임워크로, 주로 엔터프라이즈급 애플리케이션 개발에 사용됩니다. 이 프레임워크는 의존성 주입(Dependency Injection)과 관점 지향 프로그래밍(Aspect-Oriented Programming)을 통해 코드의 재사용성과 테스트 용이성을 높여줍니다. 또한, 스프링은 다양한 모듈을 제공하여 웹 애플리케이션, 데이터 접근, 보안 등 다양한 기능을 지원합니다.']


---

Parallel (병렬성)
- LCEL를 사용하여 체인을 구성할 때, 여러 체인을 동시에 실행할 수 있다.


In [22]:
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | llm
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 어느정도야?")
    | llm
    | StrOutputParser()
)

In [23]:
from langchain_core.runnables import RunnableParallel

# 병렬 실행 체인(같이 동작하는 체인)
combined = RunnableParallel(capital=chain1, area=chain2)

In [29]:
chain1.invoke({"country":"대한민국"})

'대한민국의 수도는 서울입니다.'

In [32]:
result = combined.invoke({"country":"미국"})

print(result)

{'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 면적은 약 9,830,000 평방킬로미터(약 3,796,000 평방마일)입니다. 이는 세계에서 세 번째로 큰 나라로, 러시아와 캐나다에 이어 가장 큰 면적을 가지고 있습니다.'}


In [31]:
result["capital"]
result["area"]

'대한민국의 면적은 약 100,210 평방킬로미터(㎢)입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과의 경계를 포함한 면적입니다.'

### RunnablePassThrough
- 입력을 그대로 전달하는 역할
- 단독으로 호출될 경우 단순히 입력을 받아 그대로 전달

In [34]:
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.0
)

prompt = PromptTemplate.from_template("{num}의 약수를 알려줘")

chain = prompt | llm | StrOutputParser()

In [39]:
# invoke()를 통해 실행할 때는 입력이 딕셔너리여야 하지만,
# 한 개의 변수만 템플릿에 작성되어있다면, 값만 전달해도 된다.

chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

In [43]:
from langchain_core.runnables import RunnablePassthrough

passthrough_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()

passthrough_chain.invoke({"num":10})

'10의 약수는 1, 2, 5, 10입니다.'

**RunnablePassthrough.assign()**
- 입력값으로 들어온 값의 key / value 쌍을 새롭게 할당된 key / value 쌍과 합쳐준다.

In [45]:
RunnablePassthrough.assign(new_date=lambda x : x["num"] * 3).invoke({"num": 8})

{'num': 8, 'new_date': 24}

### RunnableLambda
- 입력 값에 대한 추가 처리를 수행할 수 있다.
- 람다 함수를 사용해 입력 값에 대한 추가 처리를 진행한다.

- 날씨, 시간

In [52]:
from datetime import datetime

def get_today(num):
    print(num)
    return datetime.now().strftime("%b-%d")

# get_today()

In [53]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# prompt
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 알려줘"
)

#llm
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

#chain 생성
chain = (
    {"today": RunnableLambda(get_today), "n":itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [59]:
print(chain.invoke({"n" : 10}))

{'n': 10}
10월 28일이 생일인 유명인 중 일부는 다음과 같습니다:

1. 빌 게이츠 (Bill Gates) - 마이크로소프트의 공동 창립자
2. 줄리아 로버츠 (Julia Roberts) - 미국의 배우
3. 호아킨 피닉스 (Joaquin Phoenix) - 미국의 배우
4. 브래드 페이즐리 (Brad Paisley) - 미국의 컨트리 가수
5. 마흐무드 아흐마디네자드 (Mahmoud Ahmadinejad) - 이란의 전 대통령
6. 앤디 리치터 (Andy Richter) - 미국의 코미디언, 배우
7. 프란츠 리스트 (Franz Liszt) - 헝가리의 작곡가, 피아니스트 (1811년 출생)
8. 데이비드 워너 (David Warner) - 호주의 크리켓 선수
9. 조아킴 뢰프 (Joachim Löw) - 독일의 축구 감독
10. 치트라 바누 (Chitra Banerjee Divakaruni) - 인도의 작가

이 목록은 다양한 분야의 인물들을 포함하고 있으며, 생일은 공통적이지만 출생 연도는 다를 수 있습니다.
